<a href="https://colab.research.google.com/github/frost3242/Project_Image_Denoising/blob/main/Project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile
from google.colab import drive


zip_file_path = '/content/drive/MyDrive/Train.zip'
extract_dir = '/content/extracted_dataset'

# Extract the ZIP file
if not os.path.exists(extract_dir):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)




In [ ]:
pip install -q tensorflow numpy opencv-python

In [ ]:
pip install -q scikit-learn

In [ ]:
pip install  -q keras

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
from PIL import Image

# Define a simple convolutional neural network for super-resolution
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

In [35]:
# Set up transforms for input images (you can adjust these as needed)
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load dataset using ImageFolder
train_dataset = ImageFolder(root='/content/extracted_dataset/Train', transform=transform)

# Set up dataloader
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

# Training function
def train_model(model, criterion, optimizer, dataloader, num_epochs=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (lr_images, hr_images) in enumerate(dataloader):
            hr_images = lr_images    # Assuming LR images are stored as HR images for simplicity in ImageFolder

            lr_images = lr_images.to(device)
            hr_images = hr_images.to(device)

            optimizer.zero_grad()

            outputs = model(lr_images)
            loss = criterion(outputs, hr_images)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 10 == 9:    # Print every 10 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 10))
                running_loss = 0.0

    print('Finished Training')

In [36]:
def evaluate_model(model, criterion, dataloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    mse_loss = nn.MSELoss()
    mae_loss = nn.L1Loss()
    total_mse = 0.0
    total_psnr = 0.0
    total_mae = 0.0
    count = 0

    with torch.no_grad():
        for lr_images, _ in dataloader:
            hr_images = lr_images    # Assuming LR images are stored as HR images for simplicity in ImageFolder

            lr_images = lr_images.to(device)
            hr_images = hr_images.to(device)

            outputs = model(lr_images)
            mse = mse_loss(outputs, hr_images)
            psnr = 10 * torch.log10(1 / mse)
            mae = mae_loss(outputs, hr_images)

            total_mse += mse.item()
            total_psnr += psnr.item()
            total_mae += mae.item()
            count += 1

            print(f'Image {count}: PSNR = {psnr:.2f} dB')

    avg_mse = total_mse / count
    avg_psnr = total_psnr / count
    avg_mae = total_mae / count

    print(f'Average MSE: {avg_mse:.4f}, Average PSNR: {avg_psnr:.2f} dB, Average MAE: {avg_mae:.4f}')

    return avg_mse, avg_psnr, avg_mae

In [37]:
# Initialize model, loss function, and optimizer
model = SRCNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model on 'train' dataset
train_model(model, criterion, optimizer, train_dataloader, num_epochs=50)

# Evaluate on training dataset (optional for demonstration)
print("Evaluation on training dataset:")
evaluate_model(model, criterion, train_dataloader)

[1,    10] loss: 0.048
[1,    20] loss: 0.010
[1,    30] loss: 0.006
[2,    10] loss: 0.004
[2,    20] loss: 0.003
[2,    30] loss: 0.003
[3,    10] loss: 0.002
[3,    20] loss: 0.002
[3,    30] loss: 0.002
[4,    10] loss: 0.002
[4,    20] loss: 0.001
[4,    30] loss: 0.001
[5,    10] loss: 0.001
[5,    20] loss: 0.001
[5,    30] loss: 0.001
[6,    10] loss: 0.001
[6,    20] loss: 0.001
[6,    30] loss: 0.001
[7,    10] loss: 0.001
[7,    20] loss: 0.001
[7,    30] loss: 0.001
[8,    10] loss: 0.001
[8,    20] loss: 0.001
[8,    30] loss: 0.001
[9,    10] loss: 0.001
[9,    20] loss: 0.001
[9,    30] loss: 0.001
[10,    10] loss: 0.001
[10,    20] loss: 0.001
[10,    30] loss: 0.001
[11,    10] loss: 0.001
[11,    20] loss: 0.001
[11,    30] loss: 0.001
[12,    10] loss: 0.001
[12,    20] loss: 0.001
[12,    30] loss: 0.001
[13,    10] loss: 0.001
[13,    20] loss: 0.001
[13,    30] loss: 0.001
[14,    10] loss: 0.001
[14,    20] loss: 0.001
[14,    30] loss: 0.001
[15,    10] loss: 0

(0.0001434544190546618, 38.58813661144626, 0.006488588713710347)

In [ ]:


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [32]:
import os
class TestImageDataset(Dataset):
    def __init__(self, low_res_dir, transform=None):
        self.low_res_dir = low_res_dir
        self.low_res_files = os.listdir(low_res_dir)
        self.transform = transform

    def __len__(self):
        return len(self.low_res_files)

    def __getitem__(self, idx):
        img_name = self.low_res_files[idx]
        img_path = os.path.join(self.low_res_dir, img_name)
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image, img_name


if not os.path.exists('/content/extracted_dataset/predicted1'):
        os.makedirs('/content/extracted_dataset/predicted1')




test_dataset = TestImageDataset(low_res_dir='/content/extracted_dataset/test', transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


with torch.no_grad():
        for lr_image, img_name in test_dataloader:
            lr_image = lr_image.to(device)
            output = model(lr_image)
            output_image_path = os.path.join('/content/extracted_dataset/predicted1', img_name[0])
            save_image(output, output_image_path)

In [ ]:
def evaluate_psnr(model, criterion, dataloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    mse_loss = nn.MSELoss()
    mae_loss = nn.L1Loss()
    total_mse = 0.0
    total_psnr = 0.0
    total_mae = 0.0
    count = 0

    with torch.no_grad():
        for lr_images, _ in test_dataloader:
            hr_images = lr_images    # Assuming LR images are stored as HR images for simplicity in ImageFolder

            lr_images = lr_images.to(device)
            hr_images = hr_images.to(device)

            outputs = model(lr_images)
            mse = mse_loss(outputs, hr_images)
            psnr = 10 * torch.log10(1 / mse)
            mae = mae_loss(outputs, hr_images)

            total_mse += mse.item()
            total_psnr += psnr.item()
            total_mae += mae.item()
            count += 1

            print(f'Image {count}: PSNR = {psnr:.2f} dB')

    avg_psnr = total_psnr/count
    print(f'Average PSNR: {avg_psnr:.2f} dB')

    return avg_psnr


evaluate_psnr(model,criterion,test_dataloader)